In [1]:
import torch
from vae import TAEHVDiffusersWrapper
from text_encoder import WanTextEncoder
from diffusion import WanDiffusionWrapper
from omegaconf import OmegaConf
import os
from einops import rearrange
from torchvision.io import write_video
torch.set_grad_enabled(False)

config = OmegaConf.load('/mnt/data0/lab408/linruichen/Self-Forcing/configs/self_forcing_dmd.yaml')
default_config = OmegaConf.load("/mnt/data0/lab408/linruichen/Self-Forcing/configs/default_config.yaml")
config = OmegaConf.merge(default_config, config)

device = torch.device("musa:0")

current_vae = TAEHVDiffusersWrapper().to(device=device)
current_vae = current_vae.eval()
current_vae.to(dtype=torch.bfloat16)
current_vae.requires_grad_(False)

current_text_encoder = WanTextEncoder().to(device=device)

current_generator = WanDiffusionWrapper(**getattr(config, "model_kwargs", {}))
state_dict = torch.load('/mnt/data0/lab408/linruichen/Self-Forcing/checkpoints/self_forcing_dmd.pt', map_location=device)
current_generator.load_state_dict(state_dict['generator_ema'])
current_generator = current_generator.to(device=device)

In [2]:
from causal_inference import CausalInferencePipeline

causal_pipeline = CausalInferencePipeline(
    args=config,
    device=device,
    generator=current_generator,
    text_encoder=current_text_encoder,
    vae=current_vae
)

causal_pipeline = causal_pipeline.to(dtype=torch.bfloat16)

KV inference with 3 frames per block


In [4]:
num_output_frames=21
num_samples=1

all_video = []
prompts = ['小男孩坐在书桌前'] * num_samples

initial_latent = None
sampled_noise = torch.randn([num_samples, num_output_frames, 16, 60, 104], device=device, dtype=torch.bfloat16)

# Generate 81 frames
video, latents = causal_pipeline.inference(
    noise=sampled_noise,
    text_prompts=prompts,
    return_latents=True,
    initial_latent=initial_latent
)
current_video = rearrange(video, 'b t c h w -> b t h w c').cpu()
all_video.append(current_video)

# Final output video
video = 255.0 * torch.cat(all_video, dim=1)

os.makedirs('/mnt/data0/lab408/linruichen/Self-Forcing/videos', exist_ok=True)
for seed_idx in range(num_samples):    # All processes save their videos
    output_path = os.path.join('/mnt/data0/lab408/linruichen/Self-Forcing/videos', f'{seed_idx}_ema.mp4')
    write_video(output_path, video[seed_idx], fps=16)


current_timestep: 1000.0
current_timestep: 937.5
current_timestep: 833.3333129882812
current_timestep: 625.0
current_timestep: 1000.0
current_timestep: 937.5
current_timestep: 833.3333129882812
current_timestep: 625.0
current_timestep: 1000.0
current_timestep: 937.5
current_timestep: 833.3333129882812
current_timestep: 625.0
current_timestep: 1000.0
current_timestep: 937.5
current_timestep: 833.3333129882812
current_timestep: 625.0
current_timestep: 1000.0
current_timestep: 937.5
current_timestep: 833.3333129882812
current_timestep: 625.0
current_timestep: 1000.0
current_timestep: 937.5
current_timestep: 833.3333129882812
current_timestep: 625.0
current_timestep: 1000.0
current_timestep: 937.5
current_timestep: 833.3333129882812
current_timestep: 625.0
